In [45]:
# %pip install domolibrary --upgrade
# %pip install fastcore

In [46]:
from dataclasses import dataclass, field
from typing import List
import fnmatch
import os
import re

import importlib
import inspect

In [47]:
@dataclass
class Export:
    export_path: str

    def write_html_header(self):
        output_str = f"""
<!DOCTYPE html>
<html>
    <head>
        <title>{self.doc_title}</title>
    </head>
    <link href="prism.css" rel="stylesheet">

<body>
"""

        self._export_content(content = output_str, 
                             path = self.export_path,
                             method = 'w')

        return output_str

    def write_html_footer(self):
        output_str = """
    <script src="prism.js"></script>
    </body>
</html>"""

        self._export_content(content = output_str, 
                        path = self.export_path,
                        method = 'a')
        
        return output_str

    def _upsert_export_path(self):
        import os
        output_dir = os.path.split(self.export_path )[0] 

        if not os.path.exists(output_dir):
            print(f"creating - {output_dir}")
            os.mkdir(output_dir)
    
    @staticmethod
    def _export_content(content, path, method):
        with open(path, method, encoding = 'utf-8') as f:
            f.write(content)
        f.close()

    def _walk_dir(self):

        # https://stackoverflow.com/questions/5141437/filtering-os-walk-dirs-and-files

        # transform glob patterns to regular expressions
        include = r"|".join([fnmatch.translate(x) for x in self._include])
        exclude = r"|".join([fnmatch.translate(x) for x in self._exclude]) or r"$."

        res = []

        for root, dirs, files in os.walk(self.package_location):

            # exclude dirs
            dirs[:] = [os.path.join(root, d) for d in dirs]
            dirs[:] = [d for d in dirs if not re.match(exclude, d)]

            # exclude/include files
            files = [os.path.join(root, f) for f in files]
            files = [f for f in files if not re.match(exclude, f)]
            files = [f for f in files if re.match(include, f)]

            res += files
        return res
    


In [48]:


@dataclass
class Export_Python(Export):
    package_name : str
    
    doc_title: str = None
    
    pypi_url: str = None
    pip_install_str : str = None
    
    package_location : str = None
    package_version : str = None
   
    _exclude : List[str]= field(default_factory = lambda : ['*\__init__.py', '*\_modidx.py', '__init__.py'], repr=False)
    _include : List[str]= field(default_factory = lambda : ['*.py'], repr = False)


    def __post_init__(self):
        self.doc_title = self.doc_title or f"my {self.package_name} documentation"

        self.pypi_url = self.pypi_url or f"https://libraries.io/pypi/{self.package_name}"
        self.pip_install_str = self.pip_install_str or f"!pip install domolibrary"

        self.module = importlib.import_module(self.package_name)
        self.package_location = os.path.dirname(inspect.getfile(self.module))
        self.package_version = self.module.__version__

        self._upsert_export_path()

    
    def write_package_heading(self):
        output_str = f"""
        <div class ="packageHeading" >
            <h1>Python Package Detail:</h1>
            <p><strong>Python Package Name:</strong> " + self.package_name + "</p>
            <p><strong>Version:</strong> " + self.package_name + "</p>
            <p><strong>PyPI:</strong> <a href='{self.pypi_url}' target='_blank'>{self.pypi_url}</a></p>
            <p><strong>Pip Install:</strong> " + self.pip_install_str + "</p>
            <p><strong>Jupyter Pip Install:</strong> !pip install "{self.package_name}"</p>
            <p><strong>Location Detail:</strong> "{self.package_location}"</p>\n'
            <hr>
        </div>
        """

        self._export_content(content = output_str, 
                        path = self.export_path,
                        method = 'a')

        return output_str
    
    def write_package_structure(
        self
    ):
        output_str = f"""
        <div class = "packageStructure">
            <h2>{self.package_name} package structure:</h2>
            <ul>
        """
        
        pfiles = self._walk_dir()

        for pfile in pfiles:
            pfile_path = os.path.relpath( pfile, self.package_location)
            pfile_path = pfile_path.replace( "/", "\\" )  # Replace forward slashes with backslashes
            file_path = f"{self.package_name}\\{pfile_path}"   # Add primary folder name

            output_str += f"<li><a href=\'#{file_path}\' target='_blank'>{file_path}</a></li>\n"

        output_str +="</ul><hr></div>"

        self._export_content(content = output_str, 
                        path = self.export_path,
                        method = 'a')
        
        return output_str


    def write_file_structure(self):

        output_str = f"""
        <div class = "fileStructure">
            <h2>PY file structure and details:</h2>
        """

        pfiles = self._walk_dir()

        for pfile in pfiles:
            purl = os.path.relpath(pfile, self.package_location)
            purl = purl.replace(
                        "/", "\\"
                    )  # Replace forward slashes with backslashes
            
            output_str += f"""
            <div class = "module">
                <div class = "moduleHeading">
                    <a name='{self.package_name}\\{purl}'></a>\n"
                    <h3><a href="#top" target="_blank">HOME</a></h3>
                    <h3>Name: {self.package_name}\\{purl}</h3>
                </div>
            """

            with open(pfile, "r", encoding="utf-8") as pf:
                file_content = pf.read()
            
            file_content =  f"""
                <div class = "modulePrint">
                    <pre class = "language-python">
                        <code>
                            {file_content}
                        </code>
                    </pre><hr>
                    
                </div>
            </div>"""

            output_str += file_content

        self._export_content(content = output_str, 
                        path = self.export_path,
                        method = 'a')
        
        return output_str
    


    def export(self):
        fn_ls = [self.write_html_header, self.write_package_heading, self.write_package_structure, self.write_file_structure, self.write_html_footer]

        [fn ()for fn in fn_ls]
        print(f"exported {self.doc_title} to {self.export_path}")
        return True




In [49]:
config = Export_Python(package_name = 'domolibrary',
                export_path= 'test/test.html' )

config.__dict__

{'export_path': 'test/test.html',
 'package_name': 'domolibrary',
 'doc_title': 'my domolibrary documentation',
 'pypi_url': 'https://libraries.io/pypi/domolibrary',
 'pip_install_str': '!pip install domolibrary',
 'package_location': 'C:\\Users\\jwilson1\\AppData\\Roaming\\Python\\Python310\\site-packages\\domolibrary',
 'package_version': '4.0.34',
 '_exclude': ['*\\__init__.py', '*\\_modidx.py', '__init__.py'],
 '_include': ['*.py'],
 'module': <module 'domolibrary' from 'C:\\Users\\jwilson1\\AppData\\Roaming\\Python\\Python310\\site-packages\\domolibrary\\__init__.py'>}

In [50]:
config.write_html_header()

'\n<!DOCTYPE html>\n<html>\n    <head>\n        <title>my domolibrary documentation</title>\n    </head>\n    <link href="https://cdn.jsdelivr.net/npm/prismjs@1.29.0/themes/prism.min.css" rel="stylesheet">\n\n<body>\n'

In [51]:
config.write_package_heading()


'\n        <div class ="packageHeading" >\n            <h1>Python Package Detail:</h1>\n            <p><strong>Python Package Name:</strong> " + self.package_name + "</p>\n            <p><strong>Version:</strong> " + self.package_name + "</p>\n            <p><strong>PyPI:</strong> <a href=\'https://libraries.io/pypi/domolibrary\' target=\'_blank\'>https://libraries.io/pypi/domolibrary</a></p>\n            <p><strong>Pip Install:</strong> " + self.pip_install_str + "</p>\n            <p><strong>Jupyter Pip Install:</strong> !pip install "domolibrary"</p>\n            <p><strong>Location Detail:</strong> "C:\\Users\\jwilson1\\AppData\\Roaming\\Python\\Python310\\site-packages\\domolibrary"</p>\n\'\n            <hr>\n        </div>\n        '

In [52]:
config.write_package_structure()


'\n        <div class = "packageStructure">\n            <h2>domolibrary package structure:</h2>\n            <ul>\n        <li><a href=\'#domolibrary\\classes\\DomoAccessToken.py\' target=\'_blank\'>domolibrary\\classes\\DomoAccessToken.py</a></li>\n<li><a href=\'#domolibrary\\classes\\DomoAccount.py\' target=\'_blank\'>domolibrary\\classes\\DomoAccount.py</a></li>\n<li><a href=\'#domolibrary\\classes\\DomoAccount_Config.py\' target=\'_blank\'>domolibrary\\classes\\DomoAccount_Config.py</a></li>\n<li><a href=\'#domolibrary\\classes\\DomoActivityLog.py\' target=\'_blank\'>domolibrary\\classes\\DomoActivityLog.py</a></li>\n<li><a href=\'#domolibrary\\classes\\DomoApplication.py\' target=\'_blank\'>domolibrary\\classes\\DomoApplication.py</a></li>\n<li><a href=\'#domolibrary\\classes\\DomoApplication_Job.py\' target=\'_blank\'>domolibrary\\classes\\DomoApplication_Job.py</a></li>\n<li><a href=\'#domolibrary\\classes\\DomoBootstrap.py\' target=\'_blank\'>domolibrary\\classes\\DomoBootstra

In [53]:
config.write_file_structure()


'\n        <div class = "fileStructure">\n            <h2>PY file structure and details:</h2>\n        \n            <div class = "module">\n                <div class = "moduleHeading">\n                    <a name=\'domolibrary\\classes\\DomoAccessToken.py\'></a>\n"\n                    <h3><a href="#top" target="_blank">HOME</a></h3>\n                    <h3>Name: domolibrary\\classes\\DomoAccessToken.py</h3>\n                </div>\n            \n                <div class = "modulePrint">\n                    <pre class = "language-python">\n                        <code>\n                            # AUTOGENERATED! DO NOT EDIT! File to edit: ../../nbs/classes/50_DomoAccessToken.ipynb.\n\n# %% auto 0\n__all__ = [\'DomoAccessToken\', \'get_access_tokens\']\n\n# %% ../../nbs/classes/50_DomoAccessToken.ipynb 2\nimport httpx\nimport datetime as dt\nfrom nbdev.showdoc import patch_to\n\nfrom dataclasses import dataclass, field, asdict\n\nimport domolibrary.client.DomoAuth as dmda\nimp

In [54]:
config.write_html_footer()

'\n    <script src="https://cdn.jsdelivr.net/npm/prismjs@1.29.0/prism.min.js"></script>\n    </body>\n</html>'

In [55]:
config.export()

exported my domolibrary documentation to test/test.html


True